In [1]:
import os 
import json 
import torch
import numpy as np 
from PIL import Image
from evaluate import load
import matplotlib.pyplot as plt
from datasets import load_dataset 
from tqdm.notebook import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoProcessor, AutoModelForCausalLM,  ViTImageProcessor, AutoTokenizer

In [74]:
model_configs = {
    "blip": {
        "saved_checkpoint": "checkpoints/models/blip/",
        "processor": BlipProcessor, 
        "pretrained_path": "Salesforce/blip-image-captioning-large", 
        "inference_model": BlipForConditionalGeneration, 
        "inference_configs": {
            "temperature": 0.8,
            "top_k": 100, 
            "do_sample": True,
            "repetition_penalty": 1.2
        }
    }, 
    "git": {
        "saved_checkpoint": "checkpoints/models/git/",
        "processor": AutoProcessor, 
        "pretrained_path": "microsoft/git-base", 
        "inference_model": AutoModelForCausalLM, 
        "inference_configs": {
            "temperature": 0.8,
            "top_k": 50, 
            "do_sample": True, 
            "top_p": 0.9, 
            "repetition_penalty": 1.2
        }
    },
    "vit": {
        "saved_checkpoint": "checkpoints/models/vit/",
        "processor": [ViTImageProcessor, AutoTokenizer], 
        "pretrained_path": ["nlpconnect/vit-gpt2-image-captioning", "microsoft/git-base"], 
        "inference_model": AutoModelForCausalLM,
        "inference_configs": {
            "temperature": 0.8,
            "top_k": 50, 
            "do_sample": True
        }
    }
}

In [75]:
DATA_PATHS = {
    "annotated_set_dir": "prompts/",
    "inference_set_dir": "complete_image_set/inference/"
}

In [76]:
def load_models(model_configs):
    models = {key: {} for key in model_configs.keys()}

    for model_name in model_configs.keys(): 
        ckpt, processor, pretrained_path, inf_model, configs = model_configs[model_name].values()

        models[model_name]["model"] = inf_model.from_pretrained(ckpt) 
        models[model_name]["processor"] = [
            processor[0].from_pretrained(pretrained_path[0], kwargs="padding"), 
            processor[1].from_pretrained(pretrained_path[1], kwargs="padding")
        ] if model_name == "vit" else processor.from_pretrained(pretrained_path, kwargs="padding")
        models[model_name]["inference_configs"] = configs 
        
    return models

In [77]:
models = load_models(model_configs)

In [78]:
def load_model_comparison(file_name): 
    image_path = DATA_PATHS["inference_set_dir"] + file_name
    inference_dict = {} 
    image_processor = None 
    image = Image.open(image_path)
    
    for model_name in models.keys(): 
        model, processor, inference_configs = models[model_name].values()  
        
        if model_name == "vit": 
            image_processor, processor = processor
            
        pixel_values = image_processor(images=image, return_tensors="pt").pixel_values if image_processor else processor(images=image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values=pixel_values, max_length=200, num_return_sequences=5, **inference_configs)
        
        inference_dict[model_name] = processor.batch_decode(generated_ids, skip_special_tokens=True)

    return inference_dict

In [79]:
def print_inferences(file_name): 
    inference = load_model_comparison(file_name) 
    
    for model_name, captions in inference.items(): 
        print("Model: ", model_name.capitalize())  
    
        for caption in captions: 
            print(caption) 
    
        print()

In [80]:
print_inferences("404.png") 

Model:  Blip
attractive 404 page design with white background, black accents and a circular shape
design a simple white and grey color schemed 404 text - image graphic for a page content
design a simple white and black color palette with an elegant navigation button, followed by another section that is centered on the page, logo, and shadow based navbar
simple and elegant 404 page design with white background, black lettering & a circular illustration
create a captivating about me page design with white and black typography

Model:  Git
design an about us page with vertically stacked layout
design an elegant about me page with a white background and black font color, single section
about us page design with centered content
design a simple 404 page with an image split layout
design a simple 404 page with minimal text

Model:  Vit
create an elegant landing page design for a clothing brand
design a simple about me page with images of characters on it
minimalistic design for a landing pag

In [81]:
print_inferences("about_test.jpg") 

Model:  Blip
create an elegant landing page ui for a furniture brand with an even text - styled layout
create a landing page design for an about us page with images of interior designs and text
about me page with 3 sections, in a white and grey theme
landing page design for an architecture application with a black and white theme
create an elegant landing page design for a furniture brand with white and grey background, a play of striking images and fun graphics

Model:  Git
landing page design for educational application with card layout
design a simple about me page with minimal text and vibrant white color scheme
design a simple yet elegant about me page with an artistic look for your photography website
design an about us page with a white background, blobs of images and color fills
design a 404 page with centered text, placed on the right and left side

Model:  Vit
minimal section ui design with yellow color palette, blue background, white typography, and black geometric backgroun

In [82]:
print_inferences("about_us_test.png")

Model:  Blip
medical application landing page design with 2 sections
medical review page with 2 sections
white and yellow color scheme, modern about me page layout with 2 sections that are vertically stacked and divided into images
medical application landing page
medical application landing page

Model:  Git
create an elegant landing page design for a furniture brand with dark blue and white section backgrounds, profile image
design a simple about us page with white theme and dark color scheme
landing page design for educational application with card layouts
design an elegant about us page with minimal text
design an about me page with a 404 section that takes up most of the page, dark themed with the image and the right column

Model:  Vit
about us page design with a colorful color palette
design an a colorful 404 page with a colorful themed photo for a book with images of people in profile and a white background
design a simple about me page design with soft colors and attractive il